In [73]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 15: Science for Hungry People ---</h2><p>Today, you set out on the task of perfecting your milk-dunking cookie recipe.  All you have to do is find the right balance of ingredients.</p>
<p>Your recipe leaves room for exactly <code>100</code> teaspoons of ingredients.  You make a list of the <em>remaining ingredients you could use to finish the recipe</em> (your puzzle input) and their <em>properties per teaspoon</em>:</p>
<ul>
<li><code>capacity</code> (how well it helps the cookie absorb milk)</li>
<li><code>durability</code> (how well it keeps the cookie intact when full of milk)</li>
<li><code>flavor</code> (how tasty it makes the cookie)</li>
<li><code>texture</code> (how it improves the feel of the cookie)</li>
<li><code>calories</code> (how many calories it adds to the cookie)</li>
</ul>
<p>You can only measure ingredients in whole-teaspoon amounts accurately, and you have to be accurate so you can reproduce your results in the future.  The <em>total score</em> of a cookie can be found by adding up each of the properties (negative totals become <code>0</code>) and then multiplying together everything except calories.</p>
<p>For instance, suppose you have <span title="* I know what your preference is, but...">these two ingredients</span>:</p>
<pre><code>Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3
</code></pre>
<p>Then, choosing to use <code>44</code> teaspoons of butterscotch and <code>56</code> teaspoons of cinnamon (because the amounts of each ingredient must add up to <code>100</code>) would result in a cookie with the following properties:</p>
<ul>
<li>A <code>capacity</code> of <code>44*-1 + 56*2 = 68</code></li>
<li>A <code>durability</code> of <code>44*-2 + 56*3 = 80</code></li>
<li>A <code>flavor</code> of <code>44*6 + 56*-2 = 152</code></li>
<li>A <code>texture</code> of <code>44*3 + 56*-1 = 76</code></li>
</ul>
<p>Multiplying these together (<code>68 * 80 * 152 * 76</code>, ignoring <code>calories</code> for now) results in a total score of  <code>62842880</code>, which happens to be the best score possible given these ingredients.  If any properties had produced a negative total, it would have instead become zero, causing the whole score to multiply to zero.</p>
<p>Given the ingredients in your kitchen and their properties, what is the <em>total score</em> of the highest-scoring cookie you can make?</p>
</article>


In [74]:
from functools import cache
from typing import Iterator, Sequence


s = """
Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3
"""


@dataclass(frozen=True, slots=True)
class Ingredient:
    name: str
    capacity: int  # (how well it helps the cookie absorb milk)
    durability: int  # (how well it keeps the cookie intact when full of milk)
    flavor: int  # (how tasty it makes the cookie)
    texture: int  # (how it improves the feel of the cookie)
    calories: int  # (how many calories it adds to the cookie)


@dataclass(init=False, slots=True)
class Cookie:
    ingredients: list[Ingredient]

    def __init__(self, s: str) -> None:
        self.ingredients = self.parse(s)

    def max_score(self, total_tea_spoons: int = 100) -> int:
        return max(self.total_score(t) for t in self.ingredients_permutations())

    def max_score_with_exact_calories(
        self, total_tea_spoons: int = 100, calories: int = 500
    ) -> int:
        return max(
            self.total_score(t)
            for t in self.ingredients_permutations()
            if self.calories(t) == 500
        )

    def ingredients_permutations(
        self, total_tea_spoons: int = 100
    ) -> Iterator[tuple[int, ...]]:
        def dfs(n: int, rest: int, acc: tuple[int, ...]) -> Iterator[tuple[int, ...]]:
            if n == 1:
                yield acc + (rest,)
            else:
                for i in range(1, rest):
                    if rest - i > 0:
                        yield from dfs(n - 1, rest - i, acc + (i,))

        yield from dfs(len(self.ingredients), total_tea_spoons, tuple())

    @classmethod
    def parse(cls, s: str) -> list[Ingredient]:
        ingredients = []
        for l in s.strip().splitlines():
            name, rest = l.split(":")
            rest = [int(r.split(" ")[-1]) for r in rest.split(", ")]
            ingredients.append(Ingredient(name, *rest))
        return ingredients

    def capacity(self, tea_spones: Sequence[int]) -> int:
        return max(sum(i.capacity * t for i, t in zip(self.ingredients, tea_spones)), 0)

    def durability(self, tea_spones: Sequence[int]) -> int:
        return max(
            sum(i.durability * t for i, t in zip(self.ingredients, tea_spones)), 0
        )

    def flavor(self, tea_spones: Sequence[int]) -> int:
        return max(sum(i.flavor * t for i, t in zip(self.ingredients, tea_spones)), 0)

    def texture(self, tea_spones: Sequence[int]) -> int:
        return max(sum(i.texture * t for i, t in zip(self.ingredients, tea_spones)), 0)

    def calories(self, tea_spones: Sequence[int]) -> int:
        return max(sum(i.calories * t for i, t in zip(self.ingredients, tea_spones)), 0)

    def total_score(self, tea_spones: Sequence[int]) -> int:
        return (
            self.capacity(tea_spones)
            * self.durability(tea_spones)
            * self.flavor(tea_spones)
            * self.texture(tea_spones)
        )


assert Cookie(s).max_score() == 62842880

In [75]:
fs = """
Sprinkles: capacity 2, durability 0, flavor -2, texture 0, calories 3
Butterscotch: capacity 0, durability 5, flavor -3, texture 0, calories 3
Chocolate: capacity 0, durability 0, flavor 5, texture -1, calories 8
Candy: capacity 0, durability -1, flavor 0, texture 5, calories 8
"""
Cookie(fs).max_score()

21367368

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>21367368</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Your cookie recipe becomes wildly popular!  Someone asks if you can make another recipe that has exactly <code>500</code> calories per cookie (so they can use it as a meal replacement).  Keep the rest of your award-winning process the same (100 teaspoons, same ingredients, same scoring system).</p>
<p>For example, given the ingredients above, if you had instead selected <code>40</code> teaspoons of butterscotch and <code>60</code> teaspoons of cinnamon (which still adds to <code>100</code>), the total calorie count would be <code>40*8 + 60*3 = 500</code>.  The total score would go down, though: only <code>57600000</code>, the best you can do in such trying circumstances.</p>
<p>Given the ingredients in your kitchen and their properties, what is the <em>total score</em> of the highest-scoring cookie you can make with a calorie total of <code>500</code>?</p>
</article>

</main>


In [76]:
assert Cookie(s).max_score_with_exact_calories() == 57600000

In [77]:
Cookie(fs).max_score_with_exact_calories()

1766400

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1766400</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="15/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Science+for+Hungry+People%22+%2D+Day+15+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F15&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Science+for+Hungry+People%22+%2D+Day+15+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F15';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
